Combining the fusion.py code with the ComPat stochastic inducer, III
=======================================================
Here we will not be running the fusion code to steady state, but for dt seconds and then updating the chi.

In [ ]:
%matplotlib inline
#notebook
import numpy as np
import fusion
import ComPat
import scipy.constants  
eV = scipy.constants.eV
import os
import matplotlib
if not os.getenv("DISPLAY"): matplotlib.use('Agg')
import matplotlib.pylab as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

import scipy.constants
from fipy import Variable, FaceVariable, CellVariable, TransientTerm, DiffusionTerm, Viewer, meshes

**fusion.solve_Te** solves

$$\frac{3}{2}\frac{\partial}{\partial t}\left(n(\rho,t) T(\rho,t)\right) =
    \nabla_\rho \left[ n(\rho,t) \chi(\rho,t) \nabla_\rho
    (T(\rho,t))\right] + S(\rho, t)$$

with a boundary condition given by $Te_{bc}$ and an initial
uniform temperatore of 1000 eV; the quantities are

- $n(\rho,t)$ characterizes the plasma density

- $\chi(\rho,t)$ characterizes the thermal conductivity

- $S(\rho,t)$ characterizes the source

The geometry of the simulation is characterised by the minor radius
$a_0$, major radius $R_0$ and elongation $E_0$
(while the geometry is solved in the cylindrical approximation, the
actual radius used, $a$, is adjusted on the basis of $a_0$
and $E_0$).

These are the standard (default) settings for the fusion code except for setting dt=0.001 and plotting off.

In [ ]:
Qe_tot=2e6        # heating power [W]
H0=0              # position of Gaussian [-]
Hw=0.1            # width of Gaussian [-]
Te_bc=100         # outer edge Te boundary condition [eV]
chi=1             # thermal diffusivity [m^2 s^{-1}]
a0=1              # minor radius [m]
R0=3              # major radius [m]
E0=1.5            # ellipticity
b_pos=0.98        # position of density pedestal [-]
b_height=6e19     # height of density pedestal [m^-3]
b_sol=2e19        # sol value for density pedestal [m^-3]
b_width=0.01      # width of density pedestal [-]
b_slope=0.01      # slope of density pedestal [?]
nr=100            # number of radial grid points
dt=0.01           # time-step [s]
plots=False       # enable FiPy plots

Set up the fusion model.  We will normalise some quantities to the steady state value so we first calculate that.

In [ ]:
Te_ss, ne_ss, rho_ss, rho_norm_ss, Qe_ss, V_ss = \
    fusion.solve_Te(Qe_tot=Qe_tot, H0=H0, Hw=Hw, Te_bc=Te_bc, chi=chi, 
                    a0=a0, R0=R0, E0=E0, 
                    b_pos=b_pos, b_height=b_height, b_sol=b_sol, 
                    b_width=b_width, b_slope=b_slope, 
                    nr=nr, dt=100, plots=False)

In [ ]:
W_kin = np.cumsum(V_ss*Te_ss*ne_ss*eV*1.5)
S_e   = np.cumsum(V_ss*Qe_ss)
tau_e = W_kin / S_e
print ('Volume = %s, Stored energy = %s, Integrated source = %s, confinement time = %s' %
       (np.cumsum(V_ss)[-1], W_kin[-1], S_e[-1], tau_e[-1]))
plt.figure()
plt.plot(rho_ss, tau_e)

In [ ]:
Gd_ss = -np.gradient(Te_ss, rho_ss) * ne_ss * eV
 
Gi_ss = np.cumsum(Qe_ss *  np.diff(rho_ss).mean() * rho_ss) / (rho_ss + np.diff(rho_ss).mean()/2)
Gi_ss = np.append([Gi_ss[0]/2],(Gi_ss[1:]+Gi_ss[:-1])/2)

print(Gi_ss[Gi_ss.shape[0]//2] / Gd_ss[Gd_ss.shape[0]//2])

## Function: plot_results

In [ ]:
def plot_results(runs, run, Te_all, CASE, close_plots=False):
    # All Te profiles
    plt.figure()
    plt.plot(run['rho'], Te_all.T)
    plt.xlabel('rho')
    plt.ylabel('Te')
    plt.title('%s' % (CASE))
    plt.savefig('Te_snapshots_%s.png' % (CASE))
    if close_plots: plt.close()

    # Central Te versus time
    plt.figure()
    plt.plot(T, Te_all[:,0])
    plt.plot([T.min(),T.max()], [Te_ss[0], Te_ss[0]])
    plt.xlabel('Time [s]')
    plt.ylabel('Central Te [eV]')
    plt.title('%s' % (CASE))
    plt.savefig('Te_0_%s.png' % (CASE))
    if close_plots: plt.close()

    # chi versus time
    plt.figure()
    plt.plot(T, [r['chi'] for r in runs.values()])
    plt.plot([T.min(),T.max()], [chi, chi])
    plt.xlabel('Time [$s$]')
    plt.ylabel('$\chi$ [$m^2 s^{-1}$]')
    plt.title('%s' % (CASE))
    plt.savefig('chi_%s.png' % (CASE))
    if close_plots: plt.close()

    # Flux versus time, together with steady state "target" flux (determined by integration of the source term
    plt.figure()
    plt.plot(T, 10**np.array([r['x1'] for r in runs.values()]))
    plt.plot([T.min(),T.max()], [run['flux_ref'], run['flux_ref']])
    plt.xlabel('Time [$s$]')
    plt.ylabel('flux[$W m^{-2}$]')
    plt.title('%s\nflux mean = %0.3f, std-dev = %0.3f, ref = %0.3f' % 
              (CASE, 
               (10**np.array([r['x1'] for r in runs.values()])).mean(), 
               (10**np.array([r['x1'] for r in runs.values()])).std(),
               run['flux_ref']))
    plt.savefig('Flux_%s.png' % (CASE))
    if close_plots: plt.close()

    # Histogram of the central Te over the complete set of iterations
    plt.figure()
    plt.hist(Te_all[:,0], bins=20)
    plt.xlabel('Te[0]')
    plt.ylabel('count')
    plt.title('Histogram of Te_0 over all iterations\n%s\nmean = %0.3f, std-dev = %0.3f' % 
              (CASE, np.mean(Te_all, axis=0)[0], np.std(Te_all, axis=0)[0]))
    plt.savefig('Te_0_histogram_all_%s.png' % (CASE))
    if close_plots: plt.close()

    # Histogram of the central Te over the last 50% of the iterations
    plt.figure()
    plt.hist(Te_all[NITER//2:,0], bins=20)
    plt.xlabel('Te[0]')
    plt.ylabel('count')
    plt.title('Histogram of Te_0 over last 50%% iterations\n%s\nmean = %0.3f, std-dev = %0.3f' % 
              (CASE, np.mean(Te_all[NITER//2:], axis=0)[0], np.std(Te_all[NITER//2:], axis=0)[0]))
    plt.savefig('Te_0_histogram_last_50%%_%s.png' % (CASE))
    if close_plots: plt.close()

    # Plot of mean and standard deviation of Te over all iterations
    plt.figure()
    plt.plot(run['rho'], Te_all.mean(axis=0), 'b', label='All iterations')
    plt.fill_between(run['rho'], 
                     Te_all.mean(axis=0)-Te_all.std(axis=0), 
                     Te_all.mean(axis=0)+Te_all.std(axis=0), color='blue', alpha=0.5)
    plt.xlabel('rho')
    plt.ylabel('Te')
    plt.legend(loc=0)
    plt.title('%s' % (CASE))
    plt.savefig('Te_mean_stddev_all_%s.png' % (CASE))
    if close_plots: plt.close()

    # Plot of mean and standard deviation of Te over the last 50% of the iterations
    plt.figure()
    plt.plot(run['rho'], Te_all[NITER//2:].mean(axis=0), 'r', label='Last 50% iterations')
    plt.fill_between(run['rho'], 
                     Te_all[NITER//2:].mean(axis=0)-Te_all[NITER//2:].std(axis=0), 
                     Te_all[NITER//2:].mean(axis=0)+Te_all[NITER//2:].std(axis=0), color='red', alpha=0.5)
    plt.xlabel('rho')
    plt.ylabel('Te')
    plt.legend(loc=0)
    plt.title('%s' % (CASE))
    plt.savefig('Te_mean_stddev_last_50%%_%s.png' % (CASE))
    if close_plots: plt.close()

## Function definition: evolve_Te
Define a function to evolve **Te** for **NITER** steps of **dt** with randomize parameters of **s2**, **s3**, **d2**, **d3**, **N1**, **N2**, **N3** with targets of **l1**, **l2** and **l3** using **alpha**, **beta** and **gamma** parameters.


- the initial $\chi$ is the default (1.0)
- in each step (repeated NITER times)
    - one step of dt is made using $\chi$
    - the updated temperature profile is used to recalculate $\chi$ using a flux that has been "randomised"

$$
\chi_{new} = \frac{\Gamma_{new}}{\Gamma_{target}} * \left(\beta + (1-\beta) \left(\frac{\nabla T_{new}}{\nabla T_{target}}\right)^\gamma\right) * \chi
$$

where, for example, $\beta=0.1$ and $\gamma=4$

**Note:** that we are implementing the randomize model on the log of the fluxes!

In [ ]:
def evolve_Te(Qe_tot=2e6, H0=0, Hw=0.1, Te_bc=100, chi=1, a0=1, R0=3, E0=1.5, 
              b_pos=0.98, b_height=6e19, b_sol=2e19, b_width=0.01, b_slope=0.01, 
              nr=100, plots=False,
              dt=0.1, NITER=1000, 
              s2 = 0.2, s3 = 0.2, d2 = 10, d3 = 10, N1 = 10, N2 = 90, N3 = 30, 
              l1=1.0, l2=1.0, l3=1.0, x1=1.0, 
              alpha=0.01, beta=0.1, gamma=0.0):
    a = a0*np.sqrt(E0)
    V = 2*np.pi * 2*np.pi*R0
    mesh = meshes.CylindricalGrid1D(nr=nr, Lr=a)
    Te = CellVariable(name="Te", mesh=mesh, value=1e3)
    ne = CellVariable(name="ne", mesh=mesh, value=fusion.F_ped(mesh.cellCenters.value[0]/a, b_pos, b_height, b_sol, b_width, b_slope))
    Qe = CellVariable(name="Qe", mesh=mesh, value=np.exp(-((mesh.cellCenters.value/a-H0)/(Hw))**2)[0])
    Qe = Qe * Qe_tot/((mesh.cellVolumes*Qe.value).sum() * V)

    Gi = np.cumsum(mesh.cellVolumes*Qe.value) / (mesh.cellCenters.value[0] + mesh.dx/2)
    Gi = np.append([Gi[0]/2],(Gi[1:]+Gi[:-1])/2)

    print('Volume = %s m^3' % (mesh.cellVolumes.sum() * V))
    print('Heating power = %0.3e W' % ((mesh.cellVolumes*Qe).sum() * V))

    Te.constrain(Te_bc, mesh.facesRight)
    
    T = dt*(np.arange(NITER)+1)
    
    if plots: 
        viewer = Viewer(vars=(Te), title='Heating power = %0.3e W\nchi = %s' % (Qe.cellVolumeAverage.value * V, chi), 
                    datamin=0, datamax=5000)

    l1 = np.log10(Gi[Gi.shape[0]//2]) * l1
    l2 = np.log10(Gi[Gi.shape[0]//2]) * l2
    l3 = np.log10(Gi[Gi.shape[0]//2]) * l3
    x1 = l1; x2 = l1**2

    np.log10(Gi[Gi.shape[0]//2])
    chi_run = chi

    run = {}
    run['rho'] = mesh.cellCenters.value[0]
    run['flux_ref'] = Gi[Gi.shape[0]//2]
    
    runs = {}
    for N in np.arange(0,NITER):
        eqI = TransientTerm(coeff=scipy.constants.e*ne*1.5) == DiffusionTerm(coeff=scipy.constants.e*ne*chi_run) + Qe
        eqI.solve(var=Te, dt=dt)
        if plots: viewer.plot()
        Gd = -np.gradient(Te.value, mesh.cellCenters.value[0]) * ne.value * eV
        runs[N] = {}
        runs[N]['Te'] = Te.value.copy()
        runs[N]['x'], runs[N]['mean'], runs[N]['std'], runs[N]['xrange'], runs[N]['x1'], runs[N]['x2'] = \
            ComPat.randomize(N1=N1, N2=N2, N3=N3, 
                             l1=l1, l2=l2, l3=l3, 
                             s2=s2, s3=s3, 
                             d2=d2, d3=d3, 
                             x1=x1, x2=x2, alpha=alpha)
        runs[N]['gTnorm'] = Gd[Gd.shape[0]//2] / Gd_ss[Gd_ss.shape[0]//2]
        x1 = runs[N]['x1']
        x2 = runs[N]['x2']
        l1 = x1
        runs[N]['chi'] = (10 ** runs[N]['x1'] / Gi[Gi.shape[0]//2]) * (beta + (1.0 - beta) * runs[N]['gTnorm']**gamma) * chi
        chi_run = runs[N]['chi']
    return runs, run

## Setup and run case

Now setup the parameters, run the code and save the results

In [ ]:
dt    = 0.1                         # time-step [s]
NITER = np.int(np.rint(10.0/dt))    # all runs are for 10 seconds
T = dt*(np.arange(NITER)+1)

# randomize parameters
s2 = 0.2; s3 = 0.2
d2 = 10; d3 = 10
N1 = 10; N2 = 90; N3 = 30

# scalings for fluxes
l1 = 0.1
l2 = 1.0 
l3 = 1.0

alpha = 0.01                        # exponential averaging factor
beta  = 0.1                         # base level of chi
gamma = 4.0                         # exponent of ratio of gradients


#I=0
#for dt in [0.1, 0.01, 0.001]:
#    NITER = np.int(np.rint(10.0/dt))
#    T = dt*(np.arange(NITER)+1)
#    for alpha in [0.1, 0.01, 0.001]:
#        for gamma in [0.0, 2.0, 4.0]:
#            for S in [1e-3, 1e-2, 1e-1, 2e-1, 5e-1, 1e-0, 2e-0, 5e-0, 1e1]:
#                I += 1
#                s2 = S
#                s3 = S
#                CASE = 'dt=%s_alpha=%s_gamma=%s_s2=s3=%s' % (dt, alpha, gamma, S)
#                print(I,CASE)
                
runs, run = evolve_Te(dt = dt, NITER = NITER, 
                      s2 = s2, s3 = s3,
                      d2 = d2, d3 = d3, 
                      N1 = N1, N2 = N2, N3 = N3, 
                      l1 = l1, l2 = l2, l3 = l3,
                      alpha = alpha, beta = beta, gamma = gamma
                     )

Te_all = np.array([r['Te'] for r in runs.values()])

with open("results_test.log", "a+") as file:
    print(N1, N2, N3, s2, s3, d2, d3, alpha, beta, gamma, dt, NITER, 
          (10**np.array([r['x1'] for r in runs.values()])).mean(), 
          (10**np.array([r['x1'] for r in runs.values()])).std(),
          run['flux_ref'],
          np.mean(Te_all, axis=0)[0], np.std(Te_all, axis=0)[0],
          np.mean(Te_all[NITER//2:], axis=0)[0], np.std(Te_all[NITER//2:], axis=0)[0],
          file=file, sep=',')

plot_results(runs, run, Te_all, 
             'dt=%s_alpha=%s_gamma=%s_s2=%s_s3=%s' % (dt, alpha, gamma, s2, s3),
             False
            )